In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,13315
2,Huelva,Confirmados PDIA 14 días,1690
3,Huelva,Tasa PDIA 14 días,"330,89048699639545"
4,Huelva,Confirmados PDIA 7 días,1127
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,43
629,Municipio de Huelva sin especificar,Total Confirmados,201
630,Municipio de Huelva sin especificar,Curados,114


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  13315.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3766.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 490 personas en los últimos 7 días 

Un positivo PCR cada 316 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,13315.0,1127.0,1690.0,510743.0,220.658922,330.890487,497.0
Huelva-Costa,7661.0,684.0,1065.0,288115.0,237.405203,369.644066,291.0
Huelva (capital),3766.0,293.0,454.0,143663.0,203.949521,316.017346,133.0
Condado-Campiña,3911.0,260.0,373.0,155057.0,167.680272,240.556699,102.0
Sierra de Huelva-Andévalo Central,1543.0,140.0,192.0,67571.0,207.189475,284.145565,78.0
Isla Cristina,739.0,146.0,251.0,21264.0,686.606471,1180.398796,40.0
Aljaraque,543.0,66.0,87.0,21260.0,310.442145,409.219191,34.0
Lepe,1085.0,57.0,87.0,27431.0,207.794102,317.159418,29.0
Ayamonte,410.0,39.0,53.0,20946.0,186.193068,253.031605,25.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Encinasola,50.0,40.0,40.0,1305.0,3065.134100,3065.134100,14.0
Villalba del Alcor,185.0,71.0,80.0,3338.0,2127.022169,2396.644697,18.0
Castaño del Robledo,6.0,3.0,4.0,223.0,1345.291480,1793.721973,3.0
Isla Cristina,739.0,146.0,251.0,21264.0,686.606471,1180.398796,40.0
Calañas,64.0,14.0,19.0,2791.0,501.612325,680.759584,5.0
Cerro de Andévalo (El),68.0,9.0,16.0,2364.0,380.710660,676.818951,4.0
Cortegana,157.0,24.0,29.0,4666.0,514.359194,621.517360,9.0
Aroche,74.0,9.0,19.0,3073.0,292.873414,618.288318,2.0
Palma del Condado (La),631.0,47.0,66.0,10761.0,436.762383,613.325899,23.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
San Bartolomé de la Torre,55.0,3.0,10.0,3736.0,80.299786,267.665953,2.0,0.300000
Zarza-Perrunal (La),16.0,1.0,3.0,1267.0,78.926598,236.779795,1.0,0.333333
Corteconcepción,21.0,1.0,3.0,548.0,182.481752,547.445255,0.0,0.333333
Escacena del Campo,102.0,1.0,3.0,2284.0,43.782837,131.348511,0.0,0.333333
Valverde del Camino,292.0,4.0,10.0,12820.0,31.201248,78.003120,9.0,0.400000
Aroche,74.0,9.0,19.0,3073.0,292.873414,618.288318,2.0,0.473684
Almonaster la Real,37.0,2.0,4.0,1819.0,109.950522,219.901045,0.0,0.500000
Villablanca,72.0,2.0,4.0,2848.0,70.224719,140.449438,1.0,0.500000
Gibraleón,360.0,1.0,2.0,12607.0,7.932101,15.864202,7.0,0.500000
